In [1]:
import numpy as np
from sklearn import linear_model
import sklearn.model_selection as mdsl
from sklearn.model_selection import cross_validate

In [4]:
data2 = np.loadtxt("ds-boot-2.csv", delimiter='\t', skiprows=1)

In [5]:
def bstrapped_param(p_func, sample, B):
    n = sample.size
    strapped_samples = np.random.choice(sample, (B,n))
#     print(np.apply_along_axis(p_func, 1, strapped_samples).shape)
#     print(np.apply_along_axis(p_func, 1, strapped_samples))
    strapped_params = np.apply_along_axis(p_func, 1, strapped_samples)
    strp_params_mean = strapped_params.mean()
    strp_params_std = strapped_params.std()
    print('Evaluated parameter is {:.3f}\nIts MSE is {:.3f}'.format(strp_params_mean, strp_params_std))
#     print(np.mean(strapped_samples[1,:]))

In [6]:
B=5000

In [7]:
bstrapped_param(np.median, data2[:,-3], B)

Evaluated parameter is 8.302
Its MSE is 0.275


In [8]:
def create_B_sample_indices(dataset, B):
    m = dataset.shape[0]
    n = dataset.shape[1]
    result_indices = np.random.choice(m, (B,m))
    return result_indices

In [50]:
def train_B_regressions(dataset, B):
    indices_2d = create_B_sample_indices(dataset,B)
    m = dataset.shape[0]
    n = dataset.shape[1]-6+1
    result = np.empty((B,5,n))
#     sigmas  = np.empty(B)
    regr = linear_model.LinearRegression(normalize=True)
    i=0
    for line in indices_2d:
#         print(line)
#         print(dataset[line][:,1:-5].shape)
        regr.fit(dataset[line][:,1:-5], dataset[line][:,-5:])
        coefs = regr.coef_
#         print(coefs.shape)
        result[i] = np.insert(coefs,0, regr.intercept_, axis=1)
#         sigmas[i] = np.std(regr.predict(dataset[line][:,1:-5])[:,1]-dataset[line][:,-4])
#         result = np.append(result, coefs)
        i += 1
#     return result, sigmas
    return result

In [103]:
def analyze_coefs(coefs):
    for value in range(coefs.shape[1]):
        print('Target {}:'.format(value))
        for feature_num in range(coefs.shape[2]):
            print('\tFeature {:d}:\n\t\t95%-Confidence interval: {:.3f} +/- {:.3f}\n\t\tRange: {:.3f} - {:.3f}\n\t\t% of outliers: {:.2f}'.format(
                feature_num+1, 
                coefs[:,value,feature_num].mean(),
                coefs[:,value,feature_num].std(),
                coefs[:,value,feature_num].min(), 
                coefs[:,value,feature_num].max(), 
                coefs[:,value,feature_num][(coefs[:,value,feature_num]>3*coefs[:,value,feature_num].std()) | (coefs[:,value,feature_num]<-3*coefs[:,value,feature_num].std())].size  /B                ))

In [100]:
B=5000

In [101]:
coefs = train_B_regressions(data2, B)
analyze_coefs(coefs)

Target 0:
	Feature 1:
		95%-Confidence interval: -0.015 +/- 0.311
		Range: -9.714 - 3.118
		% of outliers: 0.00
	Feature 2:
		95%-Confidence interval: 0.336 +/- 0.518
		Range: -3.990 - 7.500
		% of outliers: 0.03
	Feature 3:
		95%-Confidence interval: 0.007 +/- 0.267
		Range: -1.601 - 4.115
		% of outliers: 0.01
	Feature 4:
		95%-Confidence interval: -0.112 +/- 0.524
		Range: -4.870 - 21.722
		% of outliers: 0.01
	Feature 5:
		95%-Confidence interval: 0.600 +/- 0.550
		Range: -18.679 - 3.835
		% of outliers: 0.02
	Feature 6:
		95%-Confidence interval: -0.282 +/- 0.381
		Range: -5.626 - 3.409
		% of outliers: 0.01
	Feature 7:
		95%-Confidence interval: 0.257 +/- 0.539
		Range: -4.486 - 11.459
		% of outliers: 0.01
	Feature 8:
		95%-Confidence interval: -0.423 +/- 0.610
		Range: -7.778 - 14.709
		% of outliers: 0.02
	Feature 9:
		95%-Confidence interval: -0.097 +/- 0.433
		Range: -3.558 - 9.572
		% of outliers: 0.01
	Feature 10:
		95%-Confidence interval: -0.038 +/- 0.433
		Range: -2.739

In [13]:
coef_no = 11

print(coefs[:,0,coef_no].mean())

print(coefs[:,0,coef_no].max())

print(coefs[:,0,coef_no].std()/coefs[:,0,coef_no].max())

0.0578465509921
11.2309787358
0.0388896195414


In [89]:
#orig
#X_t[:,0].sum()

-1.6653345369377348e-16

In [80]:
#X_t, y_t = data2[:,1:-5].copy(), data2[:,-5].copy()

In [92]:
regr = linear_model.LinearRegression(normalize=True)

regr.fit(data2[:,1:-5], data2[:,-5:])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [94]:
t_coef = np.insert(regr.coef_,0,regr.intercept_,axis=1)
# t_coef = np.insert(regr.coef_,0,regr.intercept_)
t_coef.shape

(5, 29)

In [97]:
t_coef[0]

array([-0.04561118,  0.35238786, -0.0872593 , -0.47204603,  1.12128335,
       -0.32560952,  0.91660928, -1.07016296, -0.21037968, -0.18459108,
       -0.34269019,  0.76401483,  0.04013402,  0.21971176, -0.13237054,
       -0.80100357,  0.25559588,  0.57798811, -0.12112813, -0.41820161,
       -0.25960643,  0.79350498, -0.11337128,  0.41039151, -0.3828399 ,
        0.60313631,  0.13697149, -0.39762446, -0.10356152])

In [98]:
np.std(regr.predict(data2[:,1:-5])[:,0]-data2[:,-5])

0.41803352816107664

In [17]:
def perdict(x, coef):
    return x.dot(coef)

In [55]:
t_no = 0
t_data_x = np.insert(data2[:,1:-5].copy(), 0 ,1, axis=1)
t_data_y = data2[:,t_no].copy()

In [56]:
t_data_x.shape

(40, 29)

In [60]:
np.std(perdict(t_data_x, t_coef[0])-data2[:,-5])

0.41803352816107636

In [43]:
scores = mdsl.cross_validate(regr, data2[:,1:-5], data2[:,-5], cv=5)

In [44]:
scores['test_score']

array([-30.23292015,  -0.13689516,  -1.43816752,  -2.36223534,  -1.85920685])